In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from textwrap import indent

from IPython.display import Javascript

from liblet import ANTLR, AnnotatedTreeWalker, Tree, pyast2tree, side_by_side

from operator import add, mul, truediv, sub

from operator import itemgetter

from IPython.display import Latex

from fractions import Fraction

import importlib

from core.arithmetic_manager import ArithManager

from core.util import *

In [32]:
ex1 = '1 + {2 x [(3 + 4 : 2) + 5] x 6} - [ 7 + (1 + 8) : 3 ] x [ 9 : (2 + 1) + 2 ]'
ex2 = '< 2 + 3 > / 4 / < 5 + 6 > / 7'
ex3 = '2 ^ < 3 + 4 ^ 5 : 2 ^ 7 >'
ex4 = '3 + 13 x -<1 + 3>/<2^<1+4> - 6>'
ex5a = '3 / < 4 + 2 > + 2 x { 3 : ( 4 + 5 ) + 2 ^ < 6 x 1/2 > } - < 2 x [ 5 x ( 3 + 4 ) + 5 ] - 1 > / 6'
ex5b = '3 / < 4 + 2 > + 2 x ( 3 : ( 4 + 5 ) + 2 ^ < 6 x 1/2 > ) - < 2 x ( 5 x ( 3 + 4 ) + 5 ) - 1 > / 6'

src = '(2^-1/3 + 0.5 + 8 - 2.3 x 2.1 - 2 + (((2 + 3) + 1) - 1) + (2 + (2 - 2 x (8 + 9 - (5 x 6))))):0'
expr = '( ( 96 ) ^ 2 - 87 - 52 ) : 76 / 99'


src = expr


AM = ArithManager()
ast = AM.ptree2ast(AM.ptree('antlrGrammar.g', src))
blocks = AM.blocks(ast)
MEMORY = dict(blocks)

#display(ast)

main_block = blocks[-1][1]
child_to_calc = None

current_block = block_id = None

while blocks and is_calculable(blocks[0][1]): blocks = blocks[1:]
    
while blocks:
    block_id, current_block = blocks[0]
        
    parent_to_calc = AM.prior(current_block)
    display_latex(AM.latex(main_block, MEMORY))

    parent_to_calc.children =  [Tree({'type': 'atomExpr', 'value': AM.eval(child, MEMORY), 'priority': 0, '_calc': 'last'}, []) 
                                if is_next_to_calc(child) else child
                                for child in parent_to_calc.children]
    
    if is_calculable(current_block):
        current_block = current_block.children[0]
        blocks = blocks[1:]
        
    MEMORY[block_id] = current_block

display_latex(AM.latex(main_block, {}))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [9]:
from core.shunting_yard_parser import ShuntingYardParser
from core.arithmetic_manager import ArithManager
from core.util import *

In [10]:
def ast_eval(ast, show_latex, show_ast):
    AM = ArithManager()
    
    if show_ast: display(ast)
        
    blocks = AM.blocks(ast)
    MEMORY = dict(blocks)
    main_block = blocks[-1][1]
    child_to_calc = None
    current_block = block_id = None

    while blocks and is_calculable(blocks[0][1]): blocks = blocks[1:]

    while blocks:
        block_id, current_block = blocks[0]
        parent_to_calc = AM.prior(current_block)
        tex = AM.latex(main_block, MEMORY)
        if show_latex: display_latex(tex)
        
        parent_to_calc.children =  [Tree({'type': 'atomExpr', 'value': AM.eval(child, MEMORY), 'priority': 0, '_calc': 'last'}, []) 
                                    if is_next_to_calc(child) else child
                                    for child in parent_to_calc.children]

        if is_calculable(current_block):
            current_block = current_block.children[0]
            blocks = blocks[1:]

        MEMORY[block_id] = current_block

    tex = AM.latex(main_block, {})
    if show_latex: display_latex(tex)
    
    return main_block.children[0].root['value']


def antlr_eval(expr, show_latex=False, show_ast = False):
    AM = ArithManager()
    ast = AM.ptree2ast(AM.ptree('antlrGrammar.g',expr))
    return ast_eval(ast, show_latex, show_ast)


def shunting_eval(expr, show_latex=False, show_ast = False):
    shunting_yard_parser = ShuntingYardParser()

    ast = shunting_yard_parser.parse(expr)
    return ast_eval(ast, show_latex, show_ast)


In [11]:
expr = """- ( ( ( ( ( ( 98 ) ) ) ) - ( - 41 - - - - 81 : 47 - - 52 ) : - 41 x - - 79 + 14 / 30 - - 21 + 9 / ( - ( - 97 - ( 29 - ( 73 ) ) ) ) ) ) ^ 2 - ( - - ( - 82 x - 23 - ( ( 46 - 2 ) / 93 ^ 2 ) + - ( 62 : 41 x ( - 20 + 20 ) + 16 ^ 2 x ( 59 ) ) ) ) : ( ( - - - ( 63 - ( - 27 ) / ( ( - 82 ) ) ) : ( ( ( 36 x 46 + 38 / 8 ) ) ) : - 96 x 23 - 40 - 62 + 30 / 43 + 52 / - 13 - 63 ^ 2 ^ 2 + ( ( 86 ) x - - - 23 x 10 ) ^ 2 + - - - - - 89 - ( 42 x 3 x 30 ) : ( ( 50 ) ) ) ) : - ( ( - 3 / - 28 + ( 4 - 55 ) x - ( 59 ) / - 100 / 72 ^ 2 - 93 x ( - 4 ) ) / - ( - ( 92 : 77 ) + - 63 / 41 / ( 96 ) ^ 2 - ( 24 - 40 - 9 ^ 2 ) ) x ( ( - ( 91 : 64 ) - 22 ) ) x - 38 ^ 2 : ( - 9 + - 77 + 18 ^ 2 / 20 ^ 2 x 96 - 46 ) x ( 83 : 100 ) ^ 2 x - - ( ( 79 / 37 : 18 ) - ( 2 ) ) / ( ( 38 ) ^ 2 ^ 2 + - - ( 24 ) ) : - ( ( ( - ( 43 x 4 + 77 ) - ( ( ( - ( 52 ) ) : 69 ) ) ) ) / ( - ( 70 ) : 34 ) ) x ( ( 15 : ( 27 / ( - ( 19 : 80 ) - ( 97 ) ) ) / ( 32 ) ) : - - - 95 ^ 2 / 74 : 31 x 41 x 93 + ( 46 ) / ( - 69 + 56 - 58 + ( ( 15 + 4 ) ) ) ^ 2 ) ^ 2 ) / - - ( - ( - ( 16 ) ^ 2 ^ 2 : 82 - 89 - 89 + 47 ^ 2 ) + ( - 18 : - 49 : - 68 - 56 / 85 + 31 : 84 / ( ( 53 ) ) x ( ( ( ( 78 ) ) ) ) ) - ( ( - - - 100 + ( - 17 ) x ( - - ( 10 ) ) ) ) ) - - ( ( 66 ) / 13 / 16 + 58 x ( 5 ) x - 13 ^ 2 x ( ( - - 24 / - ( 73 ^ 2 ) + 7 x - 82 ) ) + ( - - ( 46 ) / ( - 40 ) ) ) + - - ( ( 5 ) ) + ( - 92 ^ 2 : 67 : ( 63 ) ) - - ( 58 / 42 : ( 52 ) ) - - - ( 19 ) / - 29 - ( - 47 ) : ( - 5 ) ^ 2 + - ( ( - 83 + 40 : 78 / 10 ) - ( ( 38 x ( 84 ) ) ) / - ( - 36 ) + 50 ) x - - ( ( 10 ) ) / 21 ^ 2 x ( 29 ) ^ 2 ^ 2 / ( 28 : ( - 54 ) x 35 ^ 2 ) x - ( 28 ) ^ 2 + - 40 ^ 2 x 32 - 42 ^ 2 : 56 - - ( ( ( 14 / - 56 + ( 78 ) ) - - - - - 70 : 15 x ( 66 : ( 81 ) ) ^ 2 ) / - ( 86 ^ 2 + - ( - 74 ^ 2 ) ) x - - - ( 27 ) + 41 / 55 - - ( 97 ) - 92 ^ 2 ^ 2 - ( ( 88 ) ^ 2 ) x ( - 1 ) : - - 100 ^ 2 - ( 25 ^ 2 ) / ( 40 ) : ( 8 ^ 2 - 22 + 20 - 73 x - 77 + 27 : - 70 ^ 2 / 78 x ( ( 51 ) ) : ( ( - - 41 - 4 / 78 - 29 x ( 46 ) ) ) / - ( - ( 39 ) ^ 2 ) ) + ( ( ( ( ( 23 ) - - ( ( ( 73 ) ) ) ) : ( - 40 x 18 ) / ( 84 + ( 87 ) ) - ( 16 ) ) ) ) ) ^ 2 / - ( ( ( 3 ^ 2 x ( 97 ) : 69 : - 10 - ( 22 ) - - 17 ^ 2 ) x ( ( - 9 ^ 2 ) ) ) ) / ( ( ( - 13 ) + - ( 66 ) ) ) ^ 2 / - ( ( ( 42 ) : - ( ( 37 ) ) ) ) + ( - - ( 80 - 74 x 28 ^ 2 ) ^ 2 ) / - ( - 83 - 61 x - 13 ^ 2 : - 20 - 65 - - 99 / 85 / - - - 12 + - 51 ^ 2 ^ 2 + - - 8 / 8 x ( 83 ) / - 67 ^ 2 x ( - 14 ^ 2 x 49 - 52 x 27 + - 57 + 23 ) + 52 ^ 2 / 38 + 39 ^ 2 x 16 / - ( - 1 ) / 79 - 75 ^ 2 / - 96 - ( ( 52 ) : 96 ) ) + ( - ( - 84 ) - ( 100 x ( 93 ) x - - - - 15 : 12 : 40 / 67 ) : - 26 / 19 - 11 ^ 2 x - - ( 6 ) - - ( 23 ) / - 99 / - 11 + ( ( 28 ) - 37 ) ) + ( ( - - ( - ( 3 ) - - ( ( 9 ) ) / 33 ) ) ) ^ 2 x - ( - ( - ( 71 ) + 68 ) + - ( 92 ) : ( 62 ^ 2 ) + - 57 + ( - - 4 ) ^ 2 ) - - - - 24 / - 8 : ( 84 ) / - - 15 + 34 + 44 + 82 ^ 2 / ( ( - ( 91 + ( - 17 : - 100 ) ) x 23 + ( 40 ) ) ) x ( - ( ( - - ( - - 1 ^ 2 ^ 2 - ( ( 71 ) ) ) ^ 2 ) ) ) x - - ( ( - - ( ( 6 - ( 31 ) ) ) ^ 2 ) - ( - ( 8 ) x 11 ^ 2 + 82 x 100 - 88 : 52 - 33 ^ 2 : 16 / 50 ^ 2 x ( 45 ) ) ^ 2 ) : - - - - 84 / - 79 - - - 4 / 29 ^ 2 - ( 16 ) + ( 5 ^ 2 ) : - ( ( ( 18 ) / 85 : 55 + 41 ) ) x ( ( - 50 / 33 ) ) ^ 2 x - ( 22 - 8 ) : - 12 ^ 2 x ( - ( 88 / 3 ) ) + - 4 + - 90 / 47 : - - - 69 + ( 94 ) + ( - - ( 42 : 17 x 93 ) ) : - 16 + 30 / ( 20 / ( ( 52 ) ) : 41 + 8 ) + 65 ^ 2 : ( ( 88 + - 85 ) ) x - ( ( - ( 71 ) - 23 : 9 x 79 + - 26 / 59 ) x - - 14 - 92 - ( 32 - 47 + 66 ^ 2 ) - 83 : - - ( 62 ) ) x ( - - 90 / ( 84 ) x 22 / 14 ) - ( ( 24 - 61 x 85 ) ) / - ( ( 65 : ( 85 ) ) ) / ( ( 62 - 37 + ( 87 ) x 23 ) ) - - - - 30 : 4 + 9 : 38 : - ( - ( - 26 ) : 95 x 83 / 45 : 34 : 34 - - 95 - ( 22 ) x 50 - - ( ( 4 ) ) ^ 2 x 4 / 45 / ( 19 / - 45 : 81 - 98 ) : ( - ( 26 ) - ( - 69 : 25 x 12 ) ) ) x - ( - 6 x ( ( ( ( 45 ) ) ) ) - ( ( ( 1 ) ) - ( 19 ) ) ) + - ( - - - 58 - 58 / ( ( 90 ) / 5 ) ^ 2 x - 88 / ( 3 ) x ( - - ( - 10 ) ) )"""

In [42]:
from core.expression_generator import generate_expression

wrong_expr = []
for _ in range(30):
    exception = False
    expr = generate_expression(3)
    #print(expr)
    
    try:
        antlr_res = antlr_eval(expr)
    except Exception as e:
        exception = True

    try:
        shunting_res = shunting_eval(expr)
        if exception: wrong_expr.append(expr) 
    except Exception as e:
        if not exception: wrong_expr.append(expr)    

    formatted_expr = expr.replace('x', '*').replace(':', '/').replace('^', '**')
    res =  round(eval(formatted_expr), 3)

    if antlr_res != shunting_res or antlr_res - res > 1:
        wrong_expr.append(expr)
        
    
    #print(antlr_res, shunting_res, res)
        
if wrong_expr != []:
    print(f'*** WRONG {len(wrong_expr)} ***\n')
    for i, it in enumerate(wrong_expr):
        print(f' $$$ {i} $$$  ', it)
else:
    print("ALL DONE")

OverflowError: int too large to convert to float

In [35]:
expr = '( ( 96 ) ^ 2 - 87 - 52 ) : 76/99'
formatted_expr = expr.replace('x', '*').replace(':', '/').replace('^', '**')
shunting_res = shunting_eval(expr)
res =  round(eval(formatted_expr), 3)
res, shunting_res

(1.206, Fraction(898623, 76))

In [43]:
expr = '( 75 - 7 + - 14 x 35 x ( 70 ) - ( 23 x 5 : ( 40 ) ) ) / - - - 97 / - ( 43 ) / - - ( 65 ) ^ 2 x 89 : 34 x 92 : 4 + 95 + 41 / 60 ^ 2 ^ 2 - ( ( 27 - 19 : ( 7 ) - 80 : - 10 ) ) / ( 81 ) - 63 - 20 x ( ( ( ( 41 / 28 / 54 ) x 71 - 48 - 10 x 88 : 46 ^ 2 ) ) ) - ( - ( 64 ) / - - - 9 + 22 - 58 : - 48 ^ 2 ^ 2 ^ 2 ) : - - ( ( ( ( 25 / ( - 89 - 60 + 99 : 56 x 62 ) ^ 2 ) ) ) ^ 2 ) ^ 2'
antlr_res = antlr_eval(expr)

shunting_res = shunting_eval(expr)
antlr_res, shunting_res



ZeroDivisionError: Division by 0 --> 29.111:0

In [31]:
truediv(96**2-87-52, 76)

119.4342105263158